In [1]:
# Imports here
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import cv2

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [2]:
data_dir = 'facial_expressions'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
training_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])

testing_transforms = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406], 
                                                              [0.229, 0.224, 0.225])])

In [4]:
training_dataset = datasets.ImageFolder(train_dir, transform=training_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform=validation_transforms)
testing_dataset = datasets.ImageFolder(test_dir, transform=testing_transforms)

In [5]:
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True, num_workers = 2)
validate_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32)
test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=32)

In [6]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
model

Using cache found in C:\Users\turqu/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\turqu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\turqu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# Freeze pretrained model parameters to avoid backpropogating through them
for parameter in model.parameters():
    parameter.requires_grad = False


from collections import OrderedDict

# Build custom classifier
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5000)),
                                        ('relu', nn.ReLU()),
                                        ('drop', nn.Dropout(p=0.5)),
                                        ('fc2', nn.Linear(5000, 102)),
                                        ('output', nn.LogSoftmax(dim=1))]))

model.classifier = classifier

In [8]:
# Function for the validation pass
def validation(model, validateloader, criterion):
    
    val_loss = 0
    accuracy = 0
    
    for images, labels in iter(validateloader):

        images, labels = images.to('cuda'), labels.to('cuda')

        output = model.forward(images)
        val_loss += criterion(output, labels).item()

        probabilities = torch.exp(output)
        
        equality = (labels.data == probabilities.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return val_loss, accuracy

In [11]:
# Loss function and gradient descent
use_pretrained = True
save_model_path = "checkpoints/"
pth_name = "saved_model.pth"


criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

if use_pretrained:
        print('load model from %s ...' % (save_model_path+pth_name))
        checkpoint = torch.load(save_model_path+pth_name)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])

        print("epoch trained from checkpoint: " + str(checkpoint['epoch']))
        print('Done!')
#print(model.aspp.stages.c3.conv.weight)




load model from checkpoints/saved_model.pth ...


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/saved_model.pth'

In [10]:
from workspace_utils import active_session
import tqdm

def train_classifier():

    epochs = 15
    steps = 0
    print_every = 40

    model.to('cuda')
    
    for e in range(epochs):
        tq = tqdm.tqdm(total=len(train_loader)) 
        model.train()
        tq.set_description('epoch %d' % (e))
        running_loss = 0

        for images, labels in iter(train_loader):

            steps += 1

            images, labels = images.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()

            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            tq.set_postfix(loss_st='%.6f' % loss.item())
            tq.update(1)

            #if steps % print_every == 0:
        
        
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        torch.save(checkpoint, os.path.join(save_model_path, pth_name))
        print("saved the model " + save_model_path)
                
        model.eval()

        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            validation_loss, accuracy = validation(model, validate_loader, criterion)

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/print_every),
              "Validation Loss: {:.3f}.. ".format(validation_loss/len(validate_loader)),
              "Validation Accuracy: {:.3f}".format(accuracy/len(validate_loader)))

        running_loss = 0
        model.train()
        tq.close()
                    
train_classifier()

epoch 0: 100%|█████████████████████████████████████████████████████| 359/359 [03:56<00:00,  1.52it/s, loss_st=1.662353]


Epoch: 1/15..  Training Loss: 19.372..  Validation Loss: 1.405..  Validation Accuracy: 0.465


epoch 1: 100%|█████████████████████████████████████████████████████| 359/359 [04:20<00:00,  1.38it/s, loss_st=1.762907]


Epoch: 2/15..  Training Loss: 14.859..  Validation Loss: 1.389..  Validation Accuracy: 0.477


epoch 2: 100%|█████████████████████████████████████████████████████| 359/359 [04:31<00:00,  1.32it/s, loss_st=1.615968]


...

Epoch: 13/15..  Training Loss: 14.412..  Validation Loss: 1.329..  Validation Accuracy: 0.498


epoch 13: 100%|████████████████████████████████████████████████████| 359/359 [03:49<00:00,  1.56it/s, loss_st=1.690688]


Epoch: 14/15..  Training Loss: 14.358..  Validation Loss: 1.306..  Validation Accuracy: 0.515


epoch 14: 100%|████████████████████████████████████████████████████| 359/359 [05:25<00:00,  1.10it/s, loss_st=1.461166]

Epoch: 15/15..  Training Loss: 14.407..  Validation Loss: 1.337..  Validation Accuracy: 0.487


In [14]:
save_model_path = "checkpoints/"
pth_name = "saved_model.pth"
epochs = 15
checkpoint = {
    'epoch': epochs + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
}

torch.save(checkpoint, os.path.join(save_model_path, pth_name))
print("saved the model " + save_model_path)



saved the model checkpoints/
